# **Importing Libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import pandas_datareader as web
import datetime as dt

%matplotlib inline
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from matplotlib.pylab import rcParams
from matplotlib import pyplot
from fbprophet import Prophet
rcParams['figure.figsize'] = 10, 6 

ModuleNotFoundError: No module named 'fbprophet'

# **Data Collected from https://finance.yahoo.com/**

In [ ]:
import yfinance as yf
crypto_currency = 'BTC'
against_currency = 'EUR'
start = dt.datetime(2019,9,18)
end = dt.datetime(2021,10,15) #dt.datetime.now()
search = crypto_currency +'-'+ against_currency
data = yf.download(search, start=start, end=end)
data.round(2)

# **Cleaning & Transforming Data**

In [ ]:
dates_range = pd.date_range(start=start,end=end)
dates = dates_range.to_frame()
close_values = data[['Close']]
prices = dates.join(close_values)
prices = prices.reset_index()
prices = prices.drop(columns='index')
prices = prices.rename(columns={0: 'ds', 'Close': 'y'})
prices = prices.interpolate(method="backfill")

In [ ]:
prices

# **Plotting Original Data**

In [ ]:
original_mean = prices['y'].rolling(window=30).mean()

close = plt.plot(prices['y'], color='blue', label='Close Values')
mean = plt.plot(original_mean, color='red', label='Rolling Mean')
plt.grid(True)
plt.xlabel("Date")
plt.ylabel(f"Close value of {crypto_currency} in {against_currency}")
plt.legend(loc='best')

# **Predicting with Prophet Model**

In [ ]:
training_percentage = 0.80

#divide into train and validation set
train = prices[:int(training_percentage*(len(prices)))]
valid = prices[int(training_percentage*(len(prices))):]

#building the model
m = Prophet(interval_width=0.95, daily_seasonality=True, seasonality_mode='multiplicative')
model = m.fit(train)

future = m.make_future_dataframe(periods=len(valid), freq='D')
forecast_all = m.predict(future)
forecast = forecast_all[['ds', 'yhat']]
forecast = forecast.loc[(forecast['ds'] >= valid.at[len(train),'ds'])]

#forecast = pd.DataFrame(forecast,index = valid.index,columns=['Prediction'])

#calculate rmse and mape
rmse = round(sqrt(mean_squared_error(valid['y'],forecast['yhat'])),2)
mape = round(mean_absolute_percentage_error(valid['y'], forecast['yhat']),2)
print(f'RMSE = {rmse}')
print(f'MAPE = {mape}')

#plot the predictions
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=valid['ds'],
    y=valid['y'],
    name = 'Real Value',
    marker=dict(color="blue"),
    showlegend=True
))

fig.add_trace(go.Scatter(
    x=forecast['ds'],
    y=forecast['yhat'],
    name = 'Predicted Value',
    marker=dict(color="red"),
    showlegend=True
))

fig.update_layout(
    title=crypto_currency+" price prediction",
    title_x=0.5,
    xaxis_title="Date",
    yaxis_title=crypto_currency+" value in "+against_currency,
)



In [ ]:
plot1 = m.plot(forecast_all)

In [ ]:
plot2 = m.plot_components(forecast_all)